# Elasticties of the US economy
## RAND experiment
## Income elasticity

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats 
from importlib import reload
import pandas as pd
from itertools import product

In [3]:
from tools import micro
from tools import macro
from tools import params
from tools import distributions as dist
reload(micro)
reload(macro)
reload(params)
reload(dist)

<module 'tools.distributions' from '/Users/flangot/Dropbox/agghealth/notebooks/tools/distributions.py'>

## Load parmaters

In [4]:
p_com   = ['beta','psi','sigma','phi']
p_name  = ['d1','d2','p','tfp']
# loading results from estimation
pars = pd.read_pickle('../output_JPE/pars.pkl')
pars

,de,dk,fr,it,nl,se,sp,us
beta,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000
psi,0.161578,0.161578,0.161578,0.161578,0.161578,0.161578,0.161578,0.161578
sigma,2.105317,2.105317,2.105317,2.105317,2.105317,2.105317,2.105317,2.105317
phi,0.397405,0.397405,0.397405,0.397405,0.397405,0.397405,0.397405,0.397405
d1,-1.282523,-1.602088,-1.099845,-0.831846,-1.390736,-1.394792,0.002353,-0.988958
d2,4.029535,4.273080,3.786290,3.917407,3.992196,4.310991,3.434494,3.511651
p,0.836301,0.888409,0.604059,0.695076,0.657280,0.891689,0.642584,1.000000
tfp,1.011023,1.260592,0.922111,0.613212,0.989482,0.795415,0.795095,1.000000


## Benchmark simulation for GE

In [5]:
countries = ['us']
outcomes = ['m','y','c','k','n','s','csy','ksy','h','g2','g3','g4',
            'tgood','tbad','r','w','tax','oop']
results = pd.DataFrame(index=outcomes,columns=countries)
ne = 10
nk = 100
nkd = 30
size = 2* ne * nk
for co in countries: 
    opt = pd.DataFrame(index=np.arange(0,size),columns=['e','h','k','ps',
                                                        'c','m','kp','v'])
    p = pars.loc[:,co]
    theta = params.flexpars(sigma=p['sigma'],beta=p['beta'],
                            phi=p['phi'],psi=p['psi'],
                            delta_h1=p['d1'],delta_h2=p['d2'],eta=0.0,
                            tfp=p['tfp'],price=p['p'])
    # option for the numerical solution
    m  = 2.5
    op = params.settings(ne=ne,nk=nkd,maxk=190.0,curv=0.5,nprocs=40)
    inc = params.incprocess(country=co)
    inc.tauchen(ne=ne,m=m)
    aux = params.auxpars(country=co)  
    #Decision rules
    csumers = micro.bellman(options=op,flex=theta,aux=aux,inc=inc,rent=5.6e-2)
    csumers.compute_cash()
    csumers.itervalue()
    # distribution
    stats = dist.stationary(dp=csumers,nk=nk)
    stats.blowup()
    stats.compute()
    # general equilibrium
    eq = macro.equilibrium(stats=stats,taxes=False,rent=True)
    eq.solve()
    aggs = eq.aggregates()
    hlth = eq.healthreport()
    # saving aggregate outcomes
    results.loc['m',co]     = aggs.M
    results.loc['y',co]     = aggs.Y
    results.loc['c',co]     = aggs.C
    results.loc['k',co]     = aggs.K
    results.loc['n',co]     = aggs.N
    results.loc['s',co]     = p['p']*aggs.M/aggs.Y
    results.loc['csy',co]   = (aggs.C+p['p']*aggs.M)/aggs.Y
    results.loc['ksy',co]   = aggs.K/aggs.Y
    results.loc['h',co]     = hlth.pH
    results.loc['g2',co]    = hlth.gradient[0]
    results.loc['g3',co]    = hlth.gradient[1]
    results.loc['g4',co]    = hlth.gradient[2]
    results.loc['tgood',co] = hlth.pTransGood
    results.loc['tbad',co]  = hlth.pTransBad
    results.loc['r',co]     = eq.rent
    results.loc['w',co]     = eq.wage
    results.loc['tax',co]   = eq.tax
    results.loc['oop',co]   = m*p['p']*aux.copay
    # saving decision rules
    opt.loc[:,'ps'] = eq.stats.probs
    for i,s in enumerate(stats.states):
        e,h,k = s
        opt.loc[i,['e','h','k']] = [e,h,k]  
        opt.loc[i,'c'] = eq.stats.optc[e,h,k]
        opt.loc[i,'m'] = eq.stats.optm[e,h,k]
        opt.loc[i,'kp'] = eq.stats.optk[e,h,k]
        opt.loc[i,'v'] = eq.stats.value[e,h,k]

solved bellman equation (iter, diff cons, diff medexp)  250 0.015172647806789286 0.040704348040279115
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  879  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  159 0.0003945553153510417 0.0007746327012334575
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  265  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.0005 demand =  37.31615558961363 supply =  18.665607810617228  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  250 0.027999973019442903 13.715623320314327
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  1824  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.025927835051546503 demand =  18.988219712656775 supply =  144.31644508828728  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  150 0.0006539976939476944 0.0009981286035090076
grid for k (nk, maxk

In [6]:
results.loc['s','us']

0.14319574867248594

In [7]:
# loading results from estimation
momsim = pd.read_pickle('../output_JPE/momsim.pkl')
emom_us = momsim.loc[:,'us']

## Computation of the utility to be in good health

In [8]:
def means(df,var,by):
    return df.groupby(by).apply(lambda x: (x[var] * x['ps']).sum() / x['ps'].sum())

In [9]:
means(opt,'c','h')

h
0    1.185585
1    2.463233
dtype: float64

In [10]:
css = means(opt,'c','h')
css[0]

1.185585302091489

In [11]:
means(opt,'m','h')

h
0    5.920218
1    0.001142
dtype: float64

In [12]:
results.loc['h','us']

0.8958060798881737

### The value in consumption
$$ 
\frac{(c_g(1-\Delta))^{1-\sigma}}{1-\sigma} + \phi h = \frac{(c_b)^{1-\sigma}}{1-\sigma}
$$
$$
\Rightarrow \Delta = 1-\frac{\left((c_b)^{1-\sigma} - (1-\sigma)\phi h \right)^{\frac{1}{1-\sigma}}}{c_g}
$$
To obtain $c_g$ and $c_b$, we use
$$
c_g = \sum_a\sum_e c_g(a,h=1,e)\lambda(a,h=1,e),~~
c_b = \sum_a\sum_e c_g(a,h=0,e)\lambda(a,h=0,e)
$$
$h$ is the share of individual in good health at the steady state

In [13]:
ZZ1 = css[0]**(1-pars.loc['sigma','us']) 
ZZ2 = (1-pars.loc['sigma','us'])*pars.loc['phi','us']*results.loc['h','us']
ZZ3 = (ZZ1-ZZ2)**(1/(1-pars.loc['sigma','us']))
Delta = 1-ZZ3/css[1] 
Delta           

0.6613665396600829

In [14]:
results.loc['c',co]

2.3301099723173695

In [15]:
results.loc['h','us']*css[1]+(1-results.loc['h','us'])*css[0]

2.3301099723174774

In [16]:
Delta*results.loc['c',co]/css[1]

0.625623602571032

In [17]:
results.loc['h','us']

0.8958060798881737

In [18]:
results.loc['tbad','us']

0.25652820400163157

## Partial Equilibrium

In [19]:
# To check that the PE gives the same results than GE when intial guess for r is the solution of GE 
results0 = pd.DataFrame(index=outcomes,columns=countries)
for co in countries: 
    opt0 = pd.DataFrame(index=np.arange(0,size),columns=['e','h','k','ps',
                                                         'c','m','kp','v'])
    p = pars.loc[:,co]
    theta = params.flexpars(sigma=p['sigma'],beta=p['beta'],
                            phi=p['phi'],psi=p['psi'],
                            delta_h1=p['d1'],delta_h2=p['d2'],eta=0.0,
                            tfp=p['tfp'],price=p['p'])
    # option for the numerical solution
    m  = 2.5
    op = params.settings(ne=ne,nk=nkd,maxk=190.0,curv=0.5,nprocs=40)
    inc = params.incprocess(country=co)
    inc.tauchen(ne=ne,m=m)
    aux = params.auxpars(country=co)  
    #Decision rules
    csumers = micro.bellman(options=op,flex=theta,aux=aux,inc=inc,rent=results.loc['r','us'],
                                                                  taxrate=results.loc['tax','us'],
                                                                  wage=results.loc['w','us'])
    csumers.compute_cash()
    csumers.itervalue()
    # distribution
    stats = dist.stationary(dp=csumers,nk=nk)
    stats0 = stats
    stats.blowup()
    stats.compute()
    # general equilibrium
    eq = macro.equilibrium(stats=stats,inirent=results.loc['r','us'],taxes=False,rent=False)
    eq.solve()
    aggs = eq.aggregates()
    hlth = eq.healthreport()
    # saving aggregate outcomes
    results0.loc['m',co]     = aggs.M
    results0.loc['y',co]     = aggs.Y
    results0.loc['c',co]     = aggs.C
    results0.loc['k',co]     = aggs.K
    results0.loc['n',co]     = aggs.N
    results0.loc['s',co]     = p['p']*aggs.M/aggs.Y
    results0.loc['csy',co]   = (aggs.C+aggs.M)/aggs.Y
    results0.loc['ksy',co]   = aggs.K/aggs.Y
    results0.loc['h',co]     = hlth.pH
    results0.loc['g2',co]    = hlth.gradient[0]
    results0.loc['g3',co]    = hlth.gradient[1]
    results0.loc['g4',co]    = hlth.gradient[2]
    results0.loc['tgood',co] = hlth.pTransGood
    results0.loc['tbad',co]  = hlth.pTransBad
    results0.loc['r',co]     = eq.rent
    results0.loc['w',co]     = eq.wage
    results0.loc['tax',co]   = eq.tax
    results0.loc['oop',co]   = m*p['p']*aux.copay
    # saving decision rules
    opt0.loc[:,'ps'] = eq.stats.probs
    for i,s in enumerate(stats.states):
        e,h,k = s
        opt0.loc[i,['e','h','k']] = [e,h,k]  
        opt0.loc[i,'c'] = eq.stats.optc[e,h,k]
        opt0.loc[i,'m'] = eq.stats.optm[e,h,k]
        opt0.loc[i,'kp'] = eq.stats.optk[e,h,k]
        opt0.loc[i,'v'] = eq.stats.value[e,h,k]

solved bellman equation (iter, diff cons, diff medexp)  192 0.00017163525620578213 0.000956654480742003
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  335  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  1 0.00016418265528272968 0.0009947581256319893
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  1  iterations 
grid for k (nk maxk) 100 190.0


In [20]:
results0

,us
m,0.617932
y,4.314882
c,2.331649
k,28.449176
n,1.333533
s,0.143209
csy,0.683583
ksy,6.593268
h,0.895853
g2,1.135675


In [21]:
results

,us
m,0.617874
y,4.314889
c,2.33011
k,28.449289
n,1.333533
s,0.143196
csy,0.683212
ksy,6.593285
h,0.895806
g2,1.135763


## RAND Experiment - variation in copay
### Tax, wage and interest rate are given 

In [22]:
results1 = pd.DataFrame(index=outcomes,columns=countries)
for co in countries: 
    opt1 = pd.DataFrame(index=np.arange(0,size),columns=['e','h','k','ps',
                                                         'c','m','kp','v'])
    p = pars.loc[:,co]
    theta = params.flexpars(sigma=p['sigma'],beta=p['beta'],
                            phi=p['phi'],psi=p['psi'],
                            delta_h1=p['d1'],delta_h2=p['d2'],eta=0.0,
                            tfp=p['tfp'],price=p['p']*1.1)
    # option for the numerical solution
    m  = 2.5
    op = params.settings(ne=ne,nk=nkd,maxk=190.0,curv=0.5,nprocs=40)
    inc = params.incprocess(country=co)
    inc.tauchen(ne=ne,m=m)
    aux = params.auxpars(country=co)  
    #Decision rules
    csumers = micro.bellman(options=op,flex=theta,aux=aux,inc=inc,rent=results.loc['r','us'],
                                                                  taxrate=results.loc['tax','us'],
                                                                  wage=results.loc['w','us'])
    csumers.compute_cash()
    csumers.itervalue()
    # distribution
    stats = dist.stationary(dp=csumers,nk=nk)
    stats.blowup()
    stats.compute()
    # general equilibrium
    eq = macro.equilibrium(stats=stats,inirent=results.loc['r','us'],taxes=False,rent=False)
    eq.solve()
    aggs = eq.aggregates()
    hlth = eq.healthreport()
    # saving aggregate outcomes
    results1.loc['m',co]     = aggs.M
    results1.loc['y',co]     = aggs.Y
    results1.loc['c',co]     = aggs.C
    results1.loc['k',co]     = aggs.K
    results1.loc['n',co]     = aggs.N
    results1.loc['s',co]     = p['p']*aggs.M/aggs.Y
    results1.loc['csy',co]   = (aggs.C+aggs.M)/aggs.Y
    results1.loc['ksy',co]   = aggs.K/aggs.Y
    results1.loc['h',co]     = hlth.pH
    results1.loc['g2',co]    = hlth.gradient[0]
    results1.loc['g3',co]    = hlth.gradient[1]
    results1.loc['g4',co]    = hlth.gradient[2]
    results1.loc['tgood',co] = hlth.pTransGood
    results1.loc['tbad',co]  = hlth.pTransBad
    results1.loc['r',co]     = eq.rent
    results1.loc['w',co]     = eq.wage
    results1.loc['tax',co]   = eq.tax
    results1.loc['oop',co]   = m*p['p']*aux.copay
    # saving decision rules
    opt1.loc[:,'ps'] = eq.stats.probs
    for i,s in enumerate(stats.states):
        e,h,k = s
        opt1.loc[i,['e','h','k']] = [e,h,k]  
        opt1.loc[i,'c'] = eq.stats.optc[e,h,k]
        opt1.loc[i,'m'] = eq.stats.optm[e,h,k]
        opt1.loc[i,'kp'] = eq.stats.optk[e,h,k]
        opt1.loc[i,'v'] = eq.stats.value[e,h,k]

solved bellman equation (iter, diff cons, diff medexp)  190 0.00025299726354521823 0.0008703438236370431
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  304  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  1 0.0001988844090003994 0.0009330246272423182
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  1  iterations 
grid for k (nk maxk) 100 190.0


In [23]:
results1

,us
m,0.600461
y,4.304948
c,2.321434
k,28.278791
n,1.333533
s,0.139482
csy,0.67873
ksy,6.568904
h,0.891929
g2,1.140178


In [28]:
aux.copay*1.1

0.15005257142857145

In [29]:
((results1.loc['m','us']-results0.loc['m','us'])/results0.loc['m','us'])/(.1/1)

-0.28272961126158014

## Income elasticity - variation in TFP 
### Tax and interest rate are given 

In [24]:
results2 = pd.DataFrame(index=outcomes,columns=countries)
for co in countries: 
    opt2 = pd.DataFrame(index=np.arange(0,size),columns=['e','h','k','ps',
                                                         'c','m','kp','v'])
    p = pars.loc[:,co]
    theta = params.flexpars(sigma=p['sigma'],beta=p['beta'],
                            phi=p['phi'],psi=p['psi'],
                            delta_h1=p['d1'],delta_h2=p['d2'],eta=0.0,
                            tfp=p['tfp']*0.9,price=p['p'])
    # option for the numerical solution
    m  = 2.5
    op = params.settings(ne=ne,nk=nkd,maxk=190.0,curv=0.5,nprocs=40)
    inc = params.incprocess(country=co)
    inc.tauchen(ne=ne,m=m)
    aux = params.auxpars(country=co)  
    #Decision rules
    csumers = micro.bellman(options=op,flex=theta,aux=aux,inc=inc,rent=results.loc['r','us'],
                                                                  taxrate=results.loc['tax','us'],
                                                                  wage=.9*results.loc['w','us'])
    csumers.compute_cash()
    csumers.itervalue()
    # distribution
    stats = dist.stationary(dp=csumers,nk=nk)
    stats.blowup()
    stats.compute()
    # general equilibrium
    eq = macro.equilibrium(stats=stats,inirent=results.loc['r','us'],taxes=False,rent=False)
    eq.solve()
    aggs = eq.aggregates()
    hlth = eq.healthreport()
    # saving aggregate outcomes
    results2.loc['m',co]     = aggs.M
    results2.loc['y',co]     = aggs.Y
    results2.loc['c',co]     = aggs.C
    results2.loc['k',co]     = aggs.K
    results2.loc['n',co]     = aggs.N
    results2.loc['s',co]     = p['p']*aggs.M/aggs.Y
    results2.loc['csy',co]   = (aggs.C+aggs.M)/aggs.Y
    results2.loc['ksy',co]   = aggs.K/aggs.Y
    results2.loc['h',co]     = hlth.pH
    results2.loc['g2',co]    = hlth.gradient[0]
    results2.loc['g3',co]    = hlth.gradient[1]
    results2.loc['g4',co]    = hlth.gradient[2]
    results2.loc['tgood',co] = hlth.pTransGood
    results2.loc['tbad',co]  = hlth.pTransBad
    results2.loc['r',co]     = eq.rent
    results2.loc['w',co]     = eq.wage
    results2.loc['tax',co]   = eq.tax
    results2.loc['oop',co]   = m*p['p']*aux.copay
    # saving decision rules
    opt2.loc[:,'ps'] = eq.stats.probs
    for i,s in enumerate(stats.states):
        e,h,k = s
        opt2.loc[i,['e','h','k']] = [e,h,k]  
        opt2.loc[i,'c'] = eq.stats.optc[e,h,k]
        opt2.loc[i,'m'] = eq.stats.optm[e,h,k]
        opt2.loc[i,'kp'] = eq.stats.optk[e,h,k]
        opt2.loc[i,'v'] = eq.stats.value[e,h,k]

solved bellman equation (iter, diff cons, diff medexp)  176 0.0005114880101695363 0.0009148722999157144
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  330  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  250 0.05858113257081354 12.989951384442264
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  116  iterations 
grid for k (nk maxk) 100 190.0


In [25]:
results2

,us
m,0.587561
y,3.734489
c,2.013462
k,25.693071
n,1.333533
s,0.157334
csy,0.696487
ksy,6.879942
h,0.890026
g2,1.137989


In [30]:
((results2.loc['m','us']-results0.loc['m','us'])/results0.loc['m','us'])/(-.1/1)

0.4914959812606744

### Computation of the income elasticity with GE adjustments

In [26]:
results3 = pd.DataFrame(index=outcomes,columns=countries)
for co in countries: 
    opt3 = pd.DataFrame(index=np.arange(0,size),columns=['e','h','k','ps',
                                                         'c','m','kp','v'])
    p = pars.loc[:,co]
    theta = params.flexpars(sigma=p['sigma'],beta=p['beta'],
                            phi=p['phi'],psi=p['psi'],
                            delta_h1=p['d1'],delta_h2=p['d2'],eta=0.0,
                            tfp=p['tfp']*.9,price=p['p'])
    # option for the numerical solution
    m  = 2.5
    op = params.settings(ne=ne,nk=nkd,maxk=190.0,curv=0.5,nprocs=40)
    inc = params.incprocess(country=co)
    inc.tauchen(ne=ne,m=m)
    aux = params.auxpars(country=co)  
    #Decision rules
    csumers = micro.bellman(options=op,flex=theta,aux=aux,inc=inc,rent=5.6e-2)
    csumers.compute_cash()
    csumers.itervalue()
    # distribution
    stats = dist.stationary(dp=csumers,nk=nk)
    stats.blowup()
    stats.compute()
    # general equilibrium
    eq = macro.equilibrium(stats=stats,taxes=False,rent=True)
    eq.solve()
    aggs = eq.aggregates()
    hlth = eq.healthreport()
    # saving aggregate outcomes
    results3.loc['m',co]     = aggs.M
    results3.loc['y',co]     = aggs.Y
    results3.loc['c',co]     = aggs.C
    results3.loc['k',co]     = aggs.K
    results3.loc['n',co]     = aggs.N
    results3.loc['s',co]     = p['p']*aggs.M/aggs.Y
    results3.loc['csy',co]   = (aggs.C+aggs.M)/aggs.Y
    results3.loc['ksy',co]   = aggs.K/aggs.Y
    results3.loc['h',co]     = hlth.pH
    results3.loc['g2',co]    = hlth.gradient[0]
    results3.loc['g3',co]    = hlth.gradient[1]
    results3.loc['g4',co]    = hlth.gradient[2]
    results3.loc['tgood',co] = hlth.pTransGood
    results3.loc['tbad',co]  = hlth.pTransBad
    results3.loc['r',co]     = eq.rent
    results3.loc['w',co]     = eq.wage
    results3.loc['tax',co]   = eq.tax
    results3.loc['oop',co]   = m*p['p']*aux.copay
    # saving decision rules
    opt3.loc[:,'ps'] = eq.stats.probs
    for i,s in enumerate(stats.states):
        e,h,k = s
        opt3.loc[i,['e','h','k']] = [e,h,k]  
        opt3.loc[i,'c'] = eq.stats.optc[e,h,k]
        opt3.loc[i,'m'] = eq.stats.optm[e,h,k]
        opt3.loc[i,'kp'] = eq.stats.optk[e,h,k]
        opt3.loc[i,'v'] = eq.stats.value[e,h,k]

solved bellman equation (iter, diff cons, diff medexp)  250 0.015172647806789286 0.040704348040279115
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  879  iterations 
grid for k (nk maxk) 100 190.0
solved bellman equation (iter, diff cons, diff medexp)  250 0.06572524057497608 12.081620588995781
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  282  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.0005 demand =  31.452211147521766 supply =  15.63048924392928  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  140 0.00021950553181593335 0.0009180766078813463
grid for k (nk, maxk)  30 190.0
stationary distribution converged in  1901  iterations 
grid for k (nk maxk) 100 190.0
rate =  0.025927835051546503 demand =  16.00436824966629 supply =  129.10209861693565  tax =  0.20209548699654836
solved bellman equation (iter, diff cons, diff medexp)  155 0.0005955261545746993 0.0009577679425483154
grid for k (nk, maxk)

In [27]:
results3

,us
m,0.586369
y,3.663645
c,1.985748
k,24.442086
n,1.333533
s,0.160051
csy,0.702065
ksy,6.671521
h,0.884636
g2,1.148483


In [31]:
((results3.loc['m','us']-results0.loc['m','us'])/results0.loc['m','us'])/(-.1/1)

0.510784465791224

In [32]:
((results3.loc['m','us']-results.loc['m','us'])/results.loc['m','us'])/(-.1/1)

0.5098887061555416

In [33]:
p

beta     0.970000
psi      0.161578
sigma    2.105317
phi      0.397405
d1      -0.988958
d2       3.511651
p        1.000000
tfp      1.000000
Name: us, dtype: float64

In [34]:
theta.tfp

0.9